In [1]:
import base64
import time
import json
import aiohttp
import asyncio
from typing import Dict, Any
import random

In [5]:
SAMPLE_PAYLOADS = [
'''
{
  "CheckingStatus": "no_checking",
  "CreditHistory": "outstanding_credit",
  "LoanPurpose": "repairs",
  "ExistingSavings": "500_to_1000",
  "EmploymentDuration": "4_to_7",
  "Sex": "male",
  "OthersOnLoan": "co-applicant",
  "OwnsProperty": "unknown",
  "InstallmentPlans": "none",
  "Housing": "free",
  "Job": "management_self-employed",
  "Telephone": "yes",
  "ForeignWorker": "yes",
  "LoanDuration": "31",
  "LoanAmount": "8411",
  "InstallmentPercent": "5",
  "CurrentResidenceDuration": "5",
  "Age": "46",
  "ExistingCreditsCount": "2",
  "Dependents": "2"
}
''',

'''{
  "CheckingStatus": "no_checking",
  "CreditHistory": "prior_payments_delayed",
  "LoanPurpose": "repairs",
  "ExistingSavings": "less_100",
  "EmploymentDuration": "4_to_7",
  "Sex": "male",
  "OthersOnLoan": "none",
  "OwnsProperty": "unknown",
  "InstallmentPlans": "none",
  "Housing": "free",
  "Job": "management_self-employed",
  "Telephone": "yes",
  "ForeignWorker": "yes",
  "LoanDuration": "31",
  "LoanAmount": "8411",
  "InstallmentPercent": "5",
  "CurrentResidenceDuration": "5",
  "Age": "34",
  "ExistingCreditsCount": "2",
  "Dependents": "2"
}''',


'''{
  "CheckingStatus": "greater_200",
  "CreditHistory": "all_credits_paid_back",
  "LoanPurpose": "repairs",
  "ExistingSavings": "greater_1000",
  "EmploymentDuration": "4_to_7",
  "Sex": "male",
  "OthersOnLoan": "none",
  "OwnsProperty": "real_estate",
  "InstallmentPlans": "none",
  "Housing": "own",
  "Job": "skilled",
  "Telephone": "yes",
  "ForeignWorker": "yes",
  "LoanDuration": "31",
  "LoanAmount": "8411",
  "InstallmentPercent": "5",
  "CurrentResidenceDuration": "5",
  "Age": "46",
  "ExistingCreditsCount": "0",
  "Dependents": "0"
}'''
]

JSON_PAYLOADS = [json.loads(payload_str) for payload_str in SAMPLE_PAYLOADS]

URL = 'https://demo-application-user-example-com.apps.p10-openshift.rch.stglabs.ibm.com/accounts/'

In [3]:
async def predict(endpoint: str, features: Dict[str, Any]):
    # format of this request is defined by the transformer
    headers = {"Content-Type": "application/json"}
    payload = features

    # submit request async
    start = time.time()
    async with aiohttp.ClientSession() as session:
        async with session.post(endpoint, headers=headers, json=payload, ssl=False) as res:
            return {
                "response": await res.text(),
                "seconds": time.time() - start,
            }

In [9]:
random.seed(12345)


def get_features():
    while True:
        features = JSON_PAYLOADS[random.randrange(len(JSON_PAYLOADS))]
        yield features


LIST_SIZE = 16
EIGHT_MIN = 240 * 2
start = time.time()
g = get_features()
pending = [
    asyncio.create_task(predict(URL, next(g))) for _ in range(LIST_SIZE)
]
num_processed = 0
while time.time() - start < EIGHT_MIN:
    done, pending = await asyncio.wait(pending, return_when=asyncio.FIRST_COMPLETED)
    pending = pending.union(
        [asyncio.create_task(predict(URL, next(g))) for _ in range(len(done))]
    )
    num_processed += len(done)
    print(
        f"{num_processed} processed.".ljust(
            150
        ),
        end="\r",
    )